# I Thought It would be fun to turn the well-know Titanic data set into sql and analyze it

In [1197]:
import pandas as pd
from sqlalchemy import create_engine

In [1198]:
titanic = pd.read_csv('train.csv')
titanic.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [1199]:
titanic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [1200]:
#dropping Cabin Column because too much data missing
titanic = titanic.drop(columns=['Cabin'])

In [1201]:
titanic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(4)
memory usage: 76.7+ KB


In [1202]:
engine = create_engine('sqlite:///:memory:', echo=True)
conn = engine.connect()

In [1203]:
sqlite_table = "titanic_details"
titanic.to_sql(sqlite_table, conn)

2023-08-14 15:34:39,978 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-08-14 15:34:39,979 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("titanic_details")
2023-08-14 15:34:39,980 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-08-14 15:34:39,980 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("titanic_details")
2023-08-14 15:34:39,980 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-08-14 15:34:39,981 INFO sqlalchemy.engine.Engine 
CREATE TABLE titanic_details (
	"index" BIGINT, 
	"PassengerId" BIGINT, 
	"Survived" BIGINT, 
	"Pclass" BIGINT, 
	"Name" TEXT, 
	"Sex" TEXT, 
	"Age" FLOAT, 
	"SibSp" BIGINT, 
	"Parch" BIGINT, 
	"Ticket" TEXT, 
	"Fare" FLOAT, 
	"Embarked" TEXT
)


2023-08-14 15:34:39,981 INFO sqlalchemy.engine.Engine [no key 0.00024s] ()
2023-08-14 15:34:39,982 INFO sqlalchemy.engine.Engine CREATE INDEX ix_titanic_details_index ON titanic_details ("index")
2023-08-14 15:34:39,982 INFO sqlalchemy.engine.Engine [no key 0.00027s] ()
2023-08-14 15:34:39,986 INFO s

891

In [1204]:
sql = '''SELECT * FROM titanic_details'''
df = pd.read_sql_query(sql, conn)
df.head()

2023-08-14 15:34:39,993 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-08-14 15:34:39,993 INFO sqlalchemy.engine.Engine SELECT * FROM titanic_details
2023-08-14 15:34:39,994 INFO sqlalchemy.engine.Engine [raw sql] ()


,index,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked
0,0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,S
1,1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C
2,2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,S
3,3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,S
4,4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,S


# lets see what percentage of people survived the crash.

In [1205]:
sql = '''SELECT ROUND(SUM(Survived) * 100.0 / COUNT(Survived),1) as Percentage_Survived FROM titanic_details'''
df = pd.read_sql_query(sql, conn)
df

2023-08-14 15:34:40,003 INFO sqlalchemy.engine.Engine SELECT ROUND(SUM(Survived) * 100.0 / COUNT(Survived),1) as Percentage_Survived FROM titanic_details
2023-08-14 15:34:40,003 INFO sqlalchemy.engine.Engine [raw sql] ()


,Percentage_Survived
0,38.4


In [1206]:
# Going to find what percentage of people survived by different age groups
sql = '''SELECT COUNT(*) as 'Children Amount', ROUND(SUM(Survived) * 100.0 / COUNT(*),1) as 'Children Perc Survived' FROM titanic_details WHERE AGE < 18 --- 
SELECT COUNT(*) as 'Adult Amount', ROUND(SUM(Survived) * 100.0 / COUNT(*),1) as 'Adult Perc Survived' FROM titanic_details WHERE AGE > 17 AND AGE < 65  ---
SELECT COUNT(*) as 'Elderly Amount', ROUND(SUM(Survived) * 100.0 / COUNT(*),1) as 'Elderly Perc Survived' FROM titanic_details WHERE AGE > 64'''.split('---')
df = pd.concat([pd.read_sql_query(q, conn) for q in sql],axis=1)
df

2023-08-14 15:34:40,008 INFO sqlalchemy.engine.Engine SELECT COUNT(*) as 'Children Amount', ROUND(SUM(Survived) * 100.0 / COUNT(*),1) as 'Children Perc Survived' FROM titanic_details WHERE AGE < 18 
2023-08-14 15:34:40,008 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-08-14 15:34:40,009 INFO sqlalchemy.engine.Engine  
SELECT COUNT(*) as 'Adult Amount', ROUND(SUM(Survived) * 100.0 / COUNT(*),1) as 'Adult Perc Survived' FROM titanic_details WHERE AGE > 17 AND AGE < 65  
2023-08-14 15:34:40,009 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-08-14 15:34:40,010 INFO sqlalchemy.engine.Engine 
SELECT COUNT(*) as 'Elderly Amount', ROUND(SUM(Survived) * 100.0 / COUNT(*),1) as 'Elderly Perc Survived' FROM titanic_details WHERE AGE > 64
2023-08-14 15:34:40,010 INFO sqlalchemy.engine.Engine [raw sql] ()


,Children Amount,Children Perc Survived,Adult Amount,Adult Perc Survived,Elderly Amount,Elderly Perc Survived
0,113,54.0,590,38.6,11,9.1


In [1207]:
sql = '''SELECT Avg(Fare) FROM titanic_details'''
df = pd.read_sql_query(sql, conn)
df.head()

2023-08-14 15:34:40,015 INFO sqlalchemy.engine.Engine SELECT Avg(Fare) FROM titanic_details
2023-08-14 15:34:40,016 INFO sqlalchemy.engine.Engine [raw sql] ()


,Avg(Fare)
0,32.204208


In [1208]:
sql = '''SELECT Count(*) FROM titanic_details where Fare>32'''
df = pd.read_sql_query(sql, conn)
df.head()

2023-08-14 15:34:40,020 INFO sqlalchemy.engine.Engine SELECT Count(*) FROM titanic_details where Fare>32
2023-08-14 15:34:40,020 INFO sqlalchemy.engine.Engine [raw sql] ()


,Count(*)
0,211


In [1210]:
# Going to find what percentage of people survived by different fare amounts
sql = '''SELECT COUNT(*) as 'Rich_People' ,ROUND(Sum(Survived)*100.0/Count(*),1) as 'Rich Perc Survived' FROM titanic_details WHERE Fare > 32 ---
SELECT COUNT(*) as 'Poor_People' ,ROUND(Sum(Survived)*100.0/Count(*),1) as 'Poor Perc Survived' FROM titanic_details WHERE Fare< 32'''.split('---')
df = pd.concat([pd.read_sql_query(q, conn) for q in sql],axis=1)
df

2023-08-14 15:34:40,029 INFO sqlalchemy.engine.Engine SELECT COUNT(*) as 'Rich_People' ,ROUND(Sum(Survived)*100.0/Count(*),1) as 'Rich Perc Survived' FROM titanic_details WHERE Fare > 32 
2023-08-14 15:34:40,029 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-08-14 15:34:40,030 INFO sqlalchemy.engine.Engine 
SELECT COUNT(*) as 'Poor_People' ,ROUND(Sum(Survived)*100.0/Count(*),1) as 'Poor Perc Survived' FROM titanic_details WHERE Fare< 32
2023-08-14 15:34:40,030 INFO sqlalchemy.engine.Engine [raw sql] ()


,Rich_People,Rich Perc Survived,Poor_People,Poor Perc Survived
0,211,59.7,680,31.8


In [1211]:
conn.close()

2023-08-14 15:34:40,035 INFO sqlalchemy.engine.Engine ROLLBACK
